In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os
import json
import pandas as pd
import datasets

from tqdm.auto import tqdm
from glob import glob
from grapheme import graphemes
from sklearn.model_selection import GroupShuffleSplit

/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open('ToxiBenchCN/Emoji_toxic_transform.json') as f:
    emo = json.load(f)['results']

with open('ToxiBenchCN/homo_toxi_transform.json') as f:
    homo = json.load(f)['results']

with open('ToxiBenchCN/pinyin_transform.json') as f:
    pinyin = json.load(f)['results']

with open('ToxiBenchCN/radical_transform.json') as f:
    radical = json.load(f)['results']

with open('ToxiBenchCN/Swap_transform.json') as f:
    swap = json.load(f)['results']

with open('ToxiBenchCN/Traditional_transform.json') as f:
    trad = json.load(f)['results']

In [4]:
indices = []
sentences_noisy = []
sentences = []
categories = []

for datas, category in zip([emo, homo, pinyin, radical, swap, trad], ['emo', 'homo', 'pinyin', 'radical', 'swap', 'trad']):
    for data in datas:
        sentence = data['original_text']
        sentence_noisy = data['perturbed_text']
        if len(list(graphemes(sentence))) !=len(list(graphemes(sentence_noisy))):
            print('not matched')
            continue
        if sentence == sentence_noisy:
            print('not noised')
            continue
        indices.append(data['sentence_id'])
        sentences.append(data['original_text'])
        sentences_noisy.append(data['perturbed_text'])
        categories.append(category)

not noised
not matched
not noised
not matched
not noised
not noised
not noised
not matched
not matched
not noised
not noised
not matched
not matched
not noised
not noised
not noised
not noised
not noised
not matched
not noised
not matched
not matched
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not matched
not noised
not noised
not matched
not noised
not noised
not matched
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not matched
not noised
not noised
not noised
not noised
not noised
not noised
not matched
not noised
not matched
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not noised
not matched
not noised
not noised
not noised
not noised
not matched
not noised
not noised
not noised
not 

In [5]:
df = pd.DataFrame({
    'id':indices,
    'sentence': sentences,
    'sentence_noisy': sentences_noisy,
    'category': categories
})  


In [7]:
gss = GroupShuffleSplit(n_splits=1, train_size=.8, random_state=42)

In [8]:
train_indices, res_indices = next(gss.split(df, groups=df['id']))

In [9]:
train_df = df.iloc[train_indices].reset_index(drop=True)

In [10]:
res_df = df.iloc[res_indices].reset_index(drop=True)

In [11]:
gss = GroupShuffleSplit(n_splits=1, train_size=.5, random_state=42)

In [12]:
dev_indices, test_indices = next(gss.split(res_df, groups=res_df['id']))

In [13]:
dev_df = res_df.iloc[dev_indices].reset_index(drop=True)
test_df = res_df.iloc[test_indices].reset_index(drop=True)

In [14]:
train_ds = datasets.Dataset.from_pandas(train_df)
dev_ds = datasets.Dataset.from_pandas(dev_df)
test_ds = datasets.Dataset.from_pandas(test_df)

In [17]:
ds = datasets.DatasetDict({
    'train': train_ds,
    'dev': dev_ds,
    'test': test_ds
})

In [ ]:
# ds.push_to_hub(repo_id=f'jwengr/ToxiBenchCN', private=False)
ds.save_to_disk('data/ToxiBenchCN')

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.32s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/jwengr/ToxiBenchCN/commit/30481c9af8b9147c09baa7c388771c95d5c70a7c', commit_message='Upload dataset', commit_description='', oid='30481c9af8b9147c09baa7c388771c95d5c70a7c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jwengr/ToxiBenchCN', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jwengr/ToxiBenchCN'), pr_revision=None, pr_num=None)